In [1]:
import h5py
import time as t
import numpy as np
import numba as nb
from numba import njit, prange
import matplotlib.pyplot as plt

In [2]:
def hdf2dict(file_name):
    # Define the dictionary to store the datasets
    data = {}
    #file_name = 'macro421_UO2_03__900K.h5'
    with h5py.File("..//02.Macro.XS.421g/" + file_name, "r") as file:
        # Iterate over the dataset names in the group
        for dataset_name in file.keys():
            # Read the dataset
            dataset = file[dataset_name]
            # Check if the dataset is a struct
            if isinstance(dataset, h5py.Group):
                # Create a dictionary to store the struct fields
                struct_data = {}
                # Iterate over the fields in the struct
                for field_name in dataset.keys():
                    # Read the field dataset
                    field_dataset = np.array(dataset[field_name])
                    # Store the field dataset in the struct dictionary
                    struct_data[field_name] = field_dataset
                # Store the struct data in the main data dictionary
                data[dataset_name] = struct_data
            else:
                # Read the dataset as a regular array
                dataset_array = np.array(dataset)
                # Store the dataset array in the dictionary
                data[dataset_name] = dataset_array

#print(data["SigC"])
    return data

#file_name = 'macro421_UO2_03__900K.h5'
#fuelDict = hdf2dict(file_name)
#fuelDict.keys()

In [3]:
"""
import numpy as np
cimport numpy as np
cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
def perform_collision(bint virtualCollision, bint absorbed, 
                        np.ndarray[double, ndim=2] SigS, double SigA, double SigP, 
                        np.ndarray[double, ndim=1] SigTmax, 
                        np.ndarray[np.int64_t, ndim=1] iGroup, 
                        int iNeutron, 
                        np.ndarray[double, ndim=1] detectS, 
                        np.ndarray[double, ndim=1] weight, 
                        np.ndarray[double, ndim=1] fuel_chi):
    cdef double SigS_sum = np.sum(SigS)
    # ... total
    cdef double SigT = SigA + SigS_sum
    # ... virtual
    cdef double SigV = SigTmax[iGroup[iNeutron]] - SigT

    # Sample the type of the collision: virtual (do nothing) or real
    if SigV / SigTmax[iGroup[iNeutron]] >= np.random.rand():  # virtual collision

        virtualCollision = True

    else:  # real collision

        virtualCollision = False

    # Sample type of the collision: scattering or absorption
    if SigS_sum / SigT >= np.random.rand():  # isotropic scattering

        # Score scatterings with account for weight divided by the
        # total scattering cross section
        detectS[iGroup[iNeutron]] += weight[iNeutron] / SigS_sum

        # Sample the energy group of the secondary neutron
        iGroup[iNeutron] = np.argmax(np.cumsum(SigS) / SigS_sum >= np.random.rand())

    else:  # absorption

        absorbed = True

        # Neutron is converted to the new fission neutron with
        # the weight increased by eta
        weight[iNeutron] *= SigP / SigA

        # Sample the energy group for the new-born neutron
        iGroup[iNeutron] = np.argmax(np.cumsum(fuel_chi) >= np.random.rand())

    return absorbed, virtualCollision, iGroup, weight, detectS
"""

'\nimport numpy as np\ncimport numpy as np\ncimport cython\n\n@cython.boundscheck(False)\n@cython.wraparound(False)\ndef perform_collision(bint virtualCollision, bint absorbed, \n                        np.ndarray[double, ndim=2] SigS, double SigA, double SigP, \n                        np.ndarray[double, ndim=1] SigTmax, \n                        np.ndarray[np.int64_t, ndim=1] iGroup, \n                        int iNeutron, \n                        np.ndarray[double, ndim=1] detectS, \n                        np.ndarray[double, ndim=1] weight, \n                        np.ndarray[double, ndim=1] fuel_chi):\n    cdef double SigS_sum = np.sum(SigS)\n    # ... total\n    cdef double SigT = SigA + SigS_sum\n    # ... virtual\n    cdef double SigV = SigTmax[iGroup[iNeutron]] - SigT\n\n    # Sample the type of the collision: virtual (do nothing) or real\n    if SigV / SigTmax[iGroup[iNeutron]] >= np.random.rand():  # virtual collision\n\n        virtualCollision = True\n\n    else:  # real

In [4]:
"""import sample_direction
import move_neutron
import russian_roulette
import split_neutrons
import calculate_keff_cycle
import update_indices
import perform_collision

#def main():
# Start the timer
start_time = t.time()

#--------------------------------------------------------------------------
# Number of source neutrons
numNeutrons_born = 100      # INPUT

# Number of inactive source cycles to skip before starting k-eff accumulation
numCycles_inactive = 100    # INPUT

# Number of active source cycles for k-eff accumulation
numCycles_active = 2000     # INPUT

# Size of the square unit cell
pitch = 3.6  # cm           # INPUT

#--------------------------------------------------------------------------
# Path to macroscopic cross section data:
# (Assuming the corresponding data files are available and accessible)
#macro_xs_path = '..//02.Macro.XS.421g'

# Fill the structures fuel, clad, and cool with the cross-section data
fuel = hdf2dict('macro421_UO2_03__900K.h5')  # INPUT
print(f"File 'macro421_UO2_03__900K.h5' has been read in.")
clad = hdf2dict('macro421_Zry__600K.h5')     # INPUT
print(f"File 'macro421_Zry__600K.h5' has been read in.")
cool  = hdf2dict('macro421_H2OB__600K.h5')   # INPUT
print(f"File 'macro421_H2OB__600K.h5' has been read in.")

# Define the majorant: the maximum total cross-section vector
SigTmax = np.max(np.vstack((fuel["SigT"], clad["SigT"], cool["SigT"])), axis=0)

# Number of energy groups
ng = fuel['ng']

#--------------------------------------------------------------------------
# Detectors
detectS = np.zeros(ng)

#--------------------------------------------------------------------------
# Four main vectors describing the neutrons in a batch
x = np.zeros(numNeutrons_born * 2)
y = np.zeros(numNeutrons_born * 2)
weight = np.ones(numNeutrons_born * 2)
iGroup = np.ones(numNeutrons_born * 2, dtype=int)

#--------------------------------------------------------------------------
# Neutrons are assumed born randomly distributed in the cell with weight 1
# with sampled fission energy spectrum
numNeutrons = numNeutrons_born
for iNeutron in range(numNeutrons):
    x[iNeutron] = np.random.rand() * pitch
    y[iNeutron] = np.random.rand() * pitch
    weight[iNeutron] = 1
    # Sample the neutron energy group
    iGroup[iNeutron] = np.argmax(np.cumsum(fuel['chi']) >= np.random.rand()) - 1
    #print(f"iNeutron: {iNeutron}, iGroup[iNeutron]: {iGroup[iNeutron]}")

#--------------------------------------------------------------------------
# Prepare vectors for keff and standard deviation of keff
keff_expected = np.ones(numCycles_active)
sigma_keff = np.zeros(numCycles_active)
keff_active_cycle = np.ones(numCycles_active)
virtualCollision = False

# Main (power) iteration loop
for iCycle in range(1, numCycles_inactive + numCycles_active + 1):

    # Normalize the weights of the neutrons to make the total weight equal to
    # numNeutrons_born (equivalent to division by keff_cycle)
    weight = (weight / np.sum(weight, axis=0, keepdims=True)) * numNeutrons_born
    weight0 = weight.copy()
    #----------------------------------------------------------------------
    # Loop over neutrons
    for iNeutron in range(numNeutrons):

        absorbed = False

        #------------------------------------------------------------------
        # Neutron random walk cycle: from emission to absorption

        while not absorbed:

            # Sample free path length according to the Woodcock method
            freePath = -np.log(np.random.rand()) / SigTmax[iGroup[iNeutron]]
            #print(freePath)

            if not virtualCollision:
                # Sample the direction of neutron flight assuming both
                # fission and scattering are isotropic in the lab (a strong
                # assumption!)
                dirX, dirY = sample_direction.sample_direction()
            # Fly
            x, y = move_neutron.move_neutron(x, y, iNeutron, pitch, freePath, dirX, dirY)

            # Find the total and scattering cross sections
            if 0.9 < x[iNeutron] < 2.7:  # INPUT
                SigA = fuel['SigF'][iGroup[iNeutron]] + fuel['SigC'][iGroup[iNeutron]] + fuel['SigL'][iGroup[iNeutron]]
                SigS = fuel["sigS_G"]["sparse_SigS[0]"][iGroup[iNeutron], :].reshape(-1, 1)
                SigP = fuel['SigP'][0, iGroup[iNeutron]]
            elif x[iNeutron] < 0.9 or x[iNeutron] > 2.7:  # INPUT
                SigA = cool['SigC'][iGroup[iNeutron]] + cool['SigL'][iGroup[iNeutron]]
                SigS = cool["sigS_G"]["sparse_SigS[0]"][iGroup[iNeutron], :].reshape(-1, 1)
                SigP = 0
            else:
                SigA = clad['SigC'][iGroup[iNeutron]] + clad['SigL'][iGroup[iNeutron]]
                SigS = clad["sigS_G"]["sparse_SigS[0]"][iGroup[iNeutron], :].reshape(-1, 1)
                SigP = 0
            
            ## Find the other cross sections ...
            ## ... scattering
            #SigS_sum = np.sum(SigS)
            ## ... total
            #SigT = SigA + SigS_sum
            ## ... virtual
            #SigV = SigTmax[iGroup[iNeutron]] - SigT
#
            ## Sample the type of the collision: virtual (do nothing) or real
            #if SigV / SigTmax[iGroup[iNeutron]] >= np.random.rand():  # virtual collision
#
            #    virtualCollision = True
#
            #else:  # real collision
#
            #    virtualCollision = False
#
            #    # Sample type of the collision: scattering or absorption
            #    if SigS_sum / SigT >= np.random.rand():  # isotropic scattering
#
            #        # Score scatterings with account for weight divided by the
            #        # total scattering cross section
            #        detectS[iGroup[iNeutron]] += weight[iNeutron] / SigS_sum
#
            #        # Sample the energy group of the secondary neutron
            #        iGroup[iNeutron] = np.argmax(np.cumsum(SigS) / SigS_sum >= np.random.rand())
#
            #    else:  # absorption
#
            #        absorbed = True
#
            #        # Neutron is converted to the new fission neutron with
            #        # the weight increased by eta
            #        weight[iNeutron] *= SigP / SigA
#
            #        # Sample the energy group for the new-born neutron
            #        iGroup[iNeutron] = np.argmax(np.cumsum(fuel['chi']) >= np.random.rand())
            #absorbed, virtualCollision, iGroup, weight, detectS = perform_collisionNEW(virtualCollision, absorbed, SigS, SigA, SigP, SigTmax, iGroup, iNeutron, detectS, weight, fuel["chi"])
            # Reshape SigS to 1-dimensional array
            #SigS = SigS.flatten()
#
            absorbed, virtualCollision, iGroup, weight, detectS = perform_collision.perform_collision(virtualCollision, absorbed, SigS, SigA, SigP, SigTmax, iGroup, iNeutron, detectS, weight, fuel["chi"])

        # End of neutron random walk cycle: from emission to absorption
    # End of loop over neutrons
    #-------------------------------------------------------------------------------------------
    # Russian roulette
    weight = russian_roulette.russian_roulette(weight, weight0)

    #-------------------------------------------------------------------------------------------
    # Clean up absorbed or killed neutrons    
    # Convert x and y to NumPy arrays
    x = np.array(x)
    y = np.array(y)
    x, y, iGroup, weight, numNeutrons = update_indices.update_indices(x, y, iGroup, weight)

    #-------------------------------------------------------------------------------------------
    # Split too "heavy" neutrons
    weight, numNeutrons, x, y, iGroup = split_neutrons.split_neutrons(weight, numNeutrons, x, y, iGroup)

    #-------------------------------------------------------------------------------------------
    # k-eff in a cycle equals the total weight of the new generation over
    # the total weight of the old generation (the old generation weight =
    # numNeutronsBorn)
    calculate_keff_cycle.calculate_keff_cycle(iCycle, numCycles_inactive, numCycles_active, weight, weight0, 
                                              numNeutrons, keff_active_cycle, keff_expected, sigma_keff)

    # End of main (power) iteration


# Calculate the elapsed time
elapsed_time = t.time() - start_time

# Create a new HDF5 file
with h5py.File('resultsPWR.h5', 'w') as hdf:
    # Make a header for the file to be created with important parameters
    header = [
            '---------------------------------------------------------',
            'Python-based Open-source Reactor Physics Education System',
            '---------------------------------------------------------',
            '',
            'function s = resultsPWR',
            '% Results for 2D neutron transport calculation in the PWR-like unit cell using method of Monte Carlo',
            f' Number of source neutrons per k-eff cycle is {numNeutrons_born}',
            f' Number of inactive source cycles to skip before starting k-eff accumulation {numCycles_inactive}',
            f' Number of active source cycles for k-eff accumulation {numCycles_active}'
        ]

    # Write the header as attributes of the root group
    for i, line in enumerate(header):
        hdf.attrs[f'header{i}'] = line

    time = hdf.create_group("time")
    time.create_dataset("elapsedTime_(s)", data = elapsed_time)
    time.create_dataset("elapsedTime_(min)", data = elapsed_time/60)
    time.create_dataset("elapsedTime_(hrs)", data = elapsed_time/3600)

    hdf.create_dataset("keff_expected", data = keff_expected[-1])
    hdf.create_dataset("sigma", data = sigma_keff[-1])
    hdf.create_dataset("keffHistory", data = keff_expected)
    hdf.create_dataset("keffError", data = sigma_keff)

    hdf.create_dataset("eg", data = (fuel["eg"][0:ng] + fuel["eg"][1:ng+1]) / 2)

    # Calculate du and flux_du
    du = np.log(fuel["eg"][1:ng+1] / fuel["eg"][:-1])
    flux_du = detectS / du
    hdf.create_dataset("flux", data = flux_du)
    

# Plot the k-effective
plt.figure()
plt.plot(keff_expected, '-r', label='k_eff')
plt.plot(keff_expected + sigma_keff, '--b', label='k_eff +/- sigma')
plt.plot(keff_expected - sigma_keff, '--b')
plt.grid(True)
plt.xlabel('Iteration number')
plt.ylabel('k-effective')
plt.legend()
plt.tight_layout()
plt.savefig('MC_01_keff.pdf')

# Plot the spectrum
plt.figure()
plt.semilogx((fuel["eg"][0:ng] + fuel["eg"][1:ng+1]) / 2, flux_du)
plt.grid(True)
plt.xlabel('Energy, eV')
plt.ylabel('Neutron flux per unit lethargy, a.u.')
plt.tight_layout()
plt.savefig('MC_02_flux_lethargy.pdf')

    # End of function
"""

'import sample_direction\nimport move_neutron\nimport russian_roulette\nimport split_neutrons\nimport calculate_keff_cycle\nimport update_indices\nimport perform_collision\n\n#def main():\n# Start the timer\nstart_time = t.time()\n\n#--------------------------------------------------------------------------\n# Number of source neutrons\nnumNeutrons_born = 100      # INPUT\n\n# Number of inactive source cycles to skip before starting k-eff accumulation\nnumCycles_inactive = 100    # INPUT\n\n# Number of active source cycles for k-eff accumulation\nnumCycles_active = 2000     # INPUT\n\n# Size of the square unit cell\npitch = 3.6  # cm           # INPUT\n\n#--------------------------------------------------------------------------\n# Path to macroscopic cross section data:\n# (Assuming the corresponding data files are available and accessible)\n#macro_xs_path = \'..//02.Macro.XS.421g\'\n\n# Fill the structures fuel, clad, and cool with the cross-section data\nfuel = hdf2dict(\'macro421_U

In [5]:
def hdf2tuple(file_name):
    data = ()
    with h5py.File("..//02.Macro.XS.421g/" + file_name, "r") as file:
        for dataset_name in file.keys():
            dataset = file[dataset_name]
            if isinstance(dataset, h5py.Group):
                struct_data = ()
                for field_name in dataset.keys():
                    field_dataset = np.array(dataset[field_name])
                    if field_name == "sparse_Sig2":
                        struct_data += tuple(field_dataset)
                    else:
                        struct_data += (field_dataset,)
                data += (struct_data,)
            else:
                dataset_array = np.array(dataset)
                data += (dataset_array,)
    return data

#file_name = 'macro421_UO2_03__900K.h5'
#fuelTuple = hdf2tuple('macro421_UO2_03__900K.h5')
# Step 1: fuel SigF, SigC and SigL 
#fuelDict["SigF"] == fuelTuple[1]
#fuelDict["SigC"][99] == fuelTuple[0][99]
#fuelDict["SigL"] == fuelTuple[2]
#fuelDict["sigS_G"]["sparse_SigS[0]"] == fuelTuple[-2][-2]
#fuelTuple[-2][-2][0,0]
#fuelDict['SigP'] == fuelTuple[3] Furthermore: #fuelDict['SigP'][0, 99] == fuelTuple[3][0, 99]
#coolDict['SigC'] == coolTuple[0]
#coolDict['SigL'] == coolTuple[2]
#coolDict["sigS_G"]["sparse_SigS[0]"] == coolTuple[-2][-3]

#cladDict['SigC'] == cladTuple[0]
#cladDict['SigL'] == cladTuple[2]
#cladDict["sigS_G"]["sparse_SigS[0]"] == cladTuple[-2][-3]
#fuelTuple[6]

In [6]:
# Start stopwatch
start_time = t.time()  # Placeholder for stopwatch functionality in Python

#--------------------------------------------------------------------------
# Number of source neutrons
numNeutrons_born = 100      # INPUT

# Number of inactive source cycles to skip before starting k-eff accumulation
numCycles_inactive = 100    # INPUT

# Number of active source cycles for k-eff accumulation
numCycles_active = 2     # INPUT

# Size of the square unit cell
pitch = 3.6  # cm           # INPUT

# Define fuel and coolant regions
fuelLeft, fuelRight = 0.9, 2.7  # INPUT
coolLeft, coolRight = 0.7, 2.9  # INPUT
#--------------------------------------------------------------------------
# Path to macroscopic cross section data:
# (Assuming the corresponding data files are available and accessible)
macro_xs_path = '..//02.Macro.XS.421g'

# Fill the structures fuel, clad, and cool with the cross-section data
fuel = hdf2dict('macro421_UO2_03__900K.h5')  # INPUT
print(f"File 'macro421_UO2_03__900K.h5' has been read in.")
clad = hdf2dict('macro421_Zry__600K.h5')     # INPUT
print(f"File 'macro421_Zry__600K.h5' has been read in.")
cool  = hdf2dict('macro421_H2OB__600K.h5')   # INPUT
print(f"File 'macro421_H2OB__600K.h5' has been read in.")

# Define the majorant: the maximum total cross-section vector
# This part just looks at all the three different vectors and creates a new
# vector that picks out the largest value for every index from each of the 
# three vectros. Althought there are some rounding differences.
SigTmax = np.max(np.vstack((fuel["SigT"], clad["SigT"], cool["SigT"])), axis=0)

# Number of energy groups
ng = fuel['ng']

#--------------------------------------------------------------------------
# Detectors
detectS = np.zeros(ng)

#--------------------------------------------------------------------------
# Four main vectors describing the neutrons in a batch
x = np.zeros(numNeutrons_born * 2)
y = np.zeros(numNeutrons_born * 2)
weight = np.ones(numNeutrons_born * 2)
iGroup = np.ones(numNeutrons_born * 2, dtype=int)

#--------------------------------------------------------------------------
# Neutrons are assumed born randomly distributed in the cell with weight 1
# with sampled fission energy spectrum
numNeutrons = numNeutrons_born
#np.random.seed(42)  # Set the seed to 42
for iNeutron in range(numNeutrons):
    x[iNeutron] = np.random.rand() * pitch
    y[iNeutron] = np.random.rand() * pitch
    weight[iNeutron] = 1
    # Sample the neutron energy group
    iGroup[iNeutron] = np.argmax(np.cumsum(fuel['chi']) >= np.random.rand())
#print(iGroup)

#--------------------------------------------------------------------------
# Prepare vectors for keff and standard deviation of keff
keff_expected = np.ones(numCycles_active)
sigma_keff = np.zeros(numCycles_active)
keff_active_cycle = np.ones(numCycles_active)
virtualCollision = False

#fuelTuple = hdf2tuple('macro421_UO2_03__900K.h5')
#cladTuple = hdf2tuple('macro421_Zry__600K.h5')     # INPUT
#coolTuple = hdf2tuple('macro421_H2OB__600K.h5')   # INPUT

File 'macro421_UO2_03__900K.h5' has been read in.
File 'macro421_Zry__600K.h5' has been read in.
File 'macro421_H2OB__600K.h5' has been read in.


In [7]:
@nb.njit
def sample_direction():
    teta = np.pi * np.random.rand()
    phi = 2.0 * np.pi * np.random.rand()
    dirX = np.sin(teta) * np.cos(phi)
    dirY = np.sin(teta) * np.sin(phi)
    return dirX, dirY

#sample_direction()

In [8]:
@nb.njit
def move_neutron(x, y, iNeutron, pitch, freePath, dirX, dirY):
    x[iNeutron] += freePath * dirX
    y[iNeutron] += freePath * dirY

    # If outside the cell, find the corresponding point inside the cell
    while x[iNeutron] < 0:
        x[iNeutron] += pitch
    while y[iNeutron] < 0:
        y[iNeutron] += pitch
    while x[iNeutron] > pitch:
        x[iNeutron] -= pitch
    while y[iNeutron] > pitch:
        y[iNeutron] -= pitch

    return x, y

In [9]:
@nb.njit
def move_neutronV2(x, y, iNeutron, pitch, freePath, dirX, dirY):
    x[iNeutron] += freePath * dirX
    y[iNeutron] += freePath * dirY

    # Adjust neutron positions to be within the cell boundaries
    x = np.mod(x, pitch)
    y = np.mod(y, pitch)

    return x, y


In [10]:
"""dirX, dirY = sample_direction()
iNeutron = 1
np.random.seed(42)
freePath = -np.log(np.random.rand()) / SigTmax[iGroup[iNeutron]]
a = move_neutron(x, y, iNeutron, pitch, freePath, dirX, dirY) 
b = move_neutronV2(x, y, iNeutron, pitch, freePath, dirX, dirY)
a[0] == b[0]"""

'dirX, dirY = sample_direction()\niNeutron = 1\nnp.random.seed(42)\nfreePath = -np.log(np.random.rand()) / SigTmax[iGroup[iNeutron]]\na = move_neutron(x, y, iNeutron, pitch, freePath, dirX, dirY) \nb = move_neutronV2(x, y, iNeutron, pitch, freePath, dirX, dirY)\na[0] == b[0]'

In [11]:
def calculate_cross_sections(fuelLeft, fuelRight, coolLeft, coolRight, x, iNeutron, iGroup, fuel, cool, clad):
    if fuelLeft < x[iNeutron] < fuelRight:
        SigA = fuel['SigF'][iGroup[iNeutron]] + fuel['SigC'][iGroup[iNeutron]] + fuel['SigL'][iGroup[iNeutron]]
        SigS = fuel["sigS_G"]["sparse_SigS[0]"][iGroup[iNeutron], :].reshape(-1, 1)
        SigP = fuel['SigP'][0, iGroup[iNeutron]]
    elif x[iNeutron] < coolLeft or x[iNeutron] > coolRight:
        SigA = cool['SigC'][iGroup[iNeutron]] + cool['SigL'][iGroup[iNeutron]]
        SigS = cool["sigS_G"]["sparse_SigS[0]"][iGroup[iNeutron], :].reshape(-1, 1)
        SigP = 0
    else:
        SigA = clad['SigC'][iGroup[iNeutron]] + clad['SigL'][iGroup[iNeutron]]
        SigS = clad["sigS_G"]["sparse_SigS[0]"][iGroup[iNeutron], :].reshape(-1, 1)
        SigP = 0

    return SigA, SigS, SigP


In [12]:
def calculate_cross_sectionsV2(fuelLeft, fuelRight, coolLeft, coolRight, x, iNeutron, iGroup, fuel, cool, clad):

    condition1 = (fuelLeft < x) & (x < fuelRight)
    condition2 = (x < coolLeft) | (x > coolRight)

    SigA = 0.0  # Initialize SigA with zeros
    SigP = 0.0
    SigS = np.zeros_like(iGroup)  # Initialize SigA with zeros

    # Update arrays based on conditions
    SigA = np.where(condition1, fuel['SigF'][iGroup] + fuel['SigC'][iGroup] + fuel['SigL'][iGroup], SigA)
    SigA = np.where(condition2, cool['SigC'][iGroup] + cool['SigL'][iGroup], SigA)
    SigA = np.where(~(condition1 | condition2), clad['SigC'][iGroup] + clad['SigL'][iGroup], SigA)

    SigS = np.where(condition1, fuel["sigS_G"]["sparse_SigS[0]"][iGroup, :].reshape(-1, 1), SigS)
    SigS = np.where(condition2, cool["sigS_G"]["sparse_SigS[0]"][iGroup, :].reshape(-1, 1), SigS)
    SigS = np.where(~(condition1 | condition2), clad["sigS_G"]["sparse_SigS[0]"][iGroup, :].reshape(-1, 1), SigS)

    SigP = np.where(condition1, fuel['SigP'][0, iGroup], SigP)
    SigP = np.where(condition2, 0, SigP)
    
    return SigA, SigS, SigP

In [13]:
def perform_collision(virtualCollision, absorbed, SigS, SigA, SigP, SigTmax,
                      iGroup, iNeutron, detectS, weight, fuel_chi):
    SigS_sum = np.sum(SigS)
    SigT = SigA + SigS_sum
    SigV = SigTmax[iGroup[iNeutron]] - SigT

    if SigV / SigTmax[iGroup[iNeutron]] >= np.random.rand():
        virtualCollision = True
    else:
        virtualCollision = False

    if SigS_sum / SigT >= np.random.rand():
        detectS[iGroup[iNeutron]] += weight[iNeutron] / SigS_sum
        iGroup[iNeutron] = np.argmax(np.cumsum(SigS) / SigS_sum >= np.random.rand())
    else:
        absorbed = True
        weight[iNeutron] *= SigP / SigA
        iGroup[iNeutron] = np.argmax(np.cumsum(fuel_chi) >= np.random.rand())

    return absorbed, virtualCollision, iGroup, weight, detectS


In [14]:
"""@nb.njit
def simulate_collision(SigS, SigA, SigTmax, iGroup, weight, detectS, fuel_SigP, fuel_chi):
    # Find the other cross sections ...
    # ... scattering
    SigS_sum = np.sum(SigS)
    # ... total
    SigT = SigA + SigS_sum
    # ... virtual
    SigV = SigTmax[iGroup] - SigT

    # Sample the type of the collision: virtual (do nothing) or real
    if SigV / SigTmax[iGroup] >= np.random.rand():  # virtual collision
        virtualCollision = True
    else:  # real collision
        virtualCollision = False

        # Sample type of the collision: scattering or absorption
        if SigS_sum / SigT >= np.random.rand():  # isotropic scattering
            # Score scatterings with account for weight divided by the
            # total scattering cross section
            detectS[iGroup] += weight / SigS_sum

            # Sample the energy group of the secondary neutron
            iGroup = np.argmax(np.cumsum(SigS) / SigS_sum >= np.random.rand())
        else:  # absorption
            absorbed = True

            # Neutron is converted to the new fission neutron with
            # the weight increased by eta
            weight *= fuel_SigP / SigA

            # Sample the energy group for the new-born neutron
            iGroup = np.argmax(np.cumsum(fuel_chi) >= np.random.rand())

    return virtualCollision, absorbed, iGroup

weight = (weight / np.sum(weight, axis=0, keepdims=True)) * numNeutrons_born
dirX, dirY = sample_direction()
freePath = -np.log(np.random.rand()) / SigTmax[iGroup[iNeutron]]
x, y = move_neutron(x, y, iNeutron, pitch, freePath, dirX, dirY)
# Find the total and scattering cross sections
if 0.9 < x[iNeutron] < 2.7:  # INPUT
    SigA = fuel['SigF'][iGroup[iNeutron]] + fuel['SigC'][iGroup[iNeutron]] + fuel['SigL'][iGroup[iNeutron]]
    SigS = fuel["sigS_G"]["sparse_SigS[0]"][iGroup[iNeutron], :].reshape(-1, 1)
    SigP = fuel['SigP'][0, iGroup[iNeutron]]
elif x[iNeutron] < 0.9 or x[iNeutron] > 2.7:  # INPUT
    SigA = cool['SigC'][iGroup[iNeutron]] + cool['SigL'][iGroup[iNeutron]]
    SigS = cool["sigS_G"]["sparse_SigS[0]"][iGroup[iNeutron], :].reshape(-1, 1)
    SigP = 0
else:
    SigA = clad['SigC'][iGroup[iNeutron]] + clad['SigL'][iGroup[iNeutron]]
    SigS = clad["sigS_G"]["sparse_SigS[0]"][iGroup[iNeutron], :].reshape(-1, 1)
    SigP = 0

fuel_SigP = fuel["SigP"]
fuel_chi = fuel["chi"]
virtualCollision, absorbed, iGroup = simulate_collision(SigS, SigA, SigTmax, iGroup, weight, detectS, fuel_SigP, fuel_chi)
"""

'@nb.njit\ndef simulate_collision(SigS, SigA, SigTmax, iGroup, weight, detectS, fuel_SigP, fuel_chi):\n    # Find the other cross sections ...\n    # ... scattering\n    SigS_sum = np.sum(SigS)\n    # ... total\n    SigT = SigA + SigS_sum\n    # ... virtual\n    SigV = SigTmax[iGroup] - SigT\n\n    # Sample the type of the collision: virtual (do nothing) or real\n    if SigV / SigTmax[iGroup] >= np.random.rand():  # virtual collision\n        virtualCollision = True\n    else:  # real collision\n        virtualCollision = False\n\n        # Sample type of the collision: scattering or absorption\n        if SigS_sum / SigT >= np.random.rand():  # isotropic scattering\n            # Score scatterings with account for weight divided by the\n            # total scattering cross section\n            detectS[iGroup] += weight / SigS_sum\n\n            # Sample the energy group of the secondary neutron\n            iGroup = np.argmax(np.cumsum(SigS) / SigS_sum >= np.random.rand())\n        el

In [15]:
@nb.njit
def russian_roulette(weight, weight0):
    numNeutrons = len(weight)
    for iNeutron in range(numNeutrons):
        terminateP = 1 - weight[iNeutron] / weight0[iNeutron]
        if terminateP >= np.random.rand():
            weight[iNeutron] = 0  # killed
        elif terminateP > 0:
            weight[iNeutron] = weight0[iNeutron]  # restore the weight
    return weight

In [16]:
@nb.njit
def split_neutrons(weight, numNeutrons, x, y, iGroup):
    numNew = 0
    for iNeutron in range(numNeutrons):
        if weight[iNeutron] > 1:
            N = int(weight[iNeutron])
            if weight[iNeutron] - N > np.random.rand():
                N += 1
            weight[iNeutron] = weight[iNeutron] / N
            for iNew in range(N - 1):
                numNew += 1
                x = np.append(x, x[iNeutron])
                y = np.append(y, y[iNeutron])
                weight = np.append(weight, weight[iNeutron])
                iGroup = np.append(iGroup, iGroup[iNeutron])
    numNeutrons += numNew
    return weight, numNeutrons, x, y, iGroup

#weight, numNeutrons, x, y, iGroup = split_neutrons(weight, numNeutrons, x, y, iGroup)


In [17]:
@nb.njit
def split_neutronsV2(weight, numNeutrons, x, y, iGroup):
    numNew = np.sum(weight > 1)
    N = np.floor(weight).astype(int)
    N += (weight - N) > np.random.rand(len(weight))
    weight = weight / N
    numNew = int(np.sum(N - 1))

    x = np.repeat(x, N)
    y = np.repeat(y, N)
    weight = np.repeat(weight, N)
    iGroup = np.repeat(iGroup, N)

    numNeutrons += numNew
    return weight, numNeutrons, x, y, iGroup


In [18]:
@nb.njit
def update_indices(x, y, iGroup, weight):
    # Get the indices of non-zero weight
    indices = np.nonzero(weight)[0]

    # Perform indexing
    x = x[indices]
    y = y[indices]
    iGroup = iGroup[indices]
    weight = weight[indices]
    
    # Update numNeutrons
    numNeutrons = weight.shape[0]
    
    return x, y, iGroup, weight, numNeutrons

In [19]:
def calculate_keff_cycle(iCycle, numCycles_inactive, numCycles_active, weight, weight0, numNeutrons,
                         keff_active_cycle, keff_expected, sigma_keff):
    iActive = iCycle - numCycles_inactive
    keff_cycle = np.sum(weight) / np.sum(weight0)

    if iActive <= 0:
        msg = f"Inactive cycle = {iCycle:3d}/{numCycles_inactive:3d}; k-eff cycle = {keff_cycle:.5f}; numNeutrons = {numNeutrons:3d}"
        print(msg)
    else:
        keff_active_cycle[iActive-1] = keff_cycle
        keff_expected[iActive-1] = np.mean(keff_active_cycle[:iActive])
        sigma_keff[iActive-1] = np.sqrt(np.sum((keff_active_cycle[:iActive] - keff_expected[iActive-1]) ** 2) / max(iActive - 1, 1) / iActive)

        msg = f"Active cycle = {iActive:3d}/{numCycles_active:3d}; k-eff cycle = {keff_cycle:.5f}; numNeutrons = {numNeutrons:3d}; k-eff expected = {keff_expected[iActive-1]:.5f}; sigma = {sigma_keff[iActive-1]:.5f}"
        print(msg)


In [20]:
"""# Main (power) iteration loop
for iCycle in range(1, numCycles_inactive + numCycles_active + 1):

    # Normalize the weights of the neutrons to make the total weight equal to
    # numNeutrons_born (equivalent to division by keff_cycle)
    weight = (weight / np.sum(weight, axis=0, keepdims=True)) * numNeutrons_born
    weight0 = weight.copy()
    #----------------------------------------------------------------------
    # Loop over neutrons
    for iNeutron in range(numNeutrons):

        absorbed = False

        #------------------------------------------------------------------
        # Neutron random walk cycle: from emission to absorption

        while not absorbed:

            # Sample free path length according to the Woodcock method
            freePath = -np.log(np.random.rand()) / SigTmax[iGroup[iNeutron]]
            #print(freePath)

            if not virtualCollision:
                # Sample the direction of neutron flight assuming both
                # fission and scattering are isotropic in the lab (a strong
                # assumption!)
                dirX, dirY = sample_direction()
            # Fly
            x, y = move_neutron(x, y, iNeutron, pitch, freePath, dirX, dirY)

            # Find the total and scattering cross sections
            if 0.9 < x[iNeutron] < 2.7:  # INPUT
                SigA = fuel['SigF'][iGroup[iNeutron]] + fuel['SigC'][iGroup[iNeutron]] + fuel['SigL'][iGroup[iNeutron]]
                SigS = fuel["sigS_G"]["sparse_SigS[0]"][iGroup[iNeutron], :].reshape(-1, 1)
                SigP = fuel['SigP'][0, iGroup[iNeutron]]
            elif x[iNeutron] < 0.9 or x[iNeutron] > 2.7:  # INPUT
                SigA = cool['SigC'][iGroup[iNeutron]] + cool['SigL'][iGroup[iNeutron]]
                SigS = cool["sigS_G"]["sparse_SigS[0]"][iGroup[iNeutron], :].reshape(-1, 1)
                SigP = 0
            else:
                SigA = clad['SigC'][iGroup[iNeutron]] + clad['SigL'][iGroup[iNeutron]]
                SigS = clad["sigS_G"]["sparse_SigS[0]"][iGroup[iNeutron], :].reshape(-1, 1)
                SigP = 0

            # Find the other cross sections ...
            # ... scattering
            SigS_sum = np.sum(SigS)
            # ... total
            SigT = SigA + SigS_sum
            # ... virtual
            SigV = SigTmax[iGroup[iNeutron]] - SigT

            # Sample the type of the collision: virtual (do nothing) or real
            if SigV / SigTmax[iGroup[iNeutron]] >= np.random.rand():  # virtual collision

                virtualCollision = True

            else:  # real collision

                virtualCollision = False

                # Sample type of the collision: scattering or absorption
                if SigS_sum / SigT >= np.random.rand():  # isotropic scattering

                    # Score scatterings with account for weight divided by the
                    # total scattering cross section
                    detectS[iGroup[iNeutron]] += weight[iNeutron] / SigS_sum

                    # Sample the energy group of the secondary neutron
                    iGroup[iNeutron] = np.argmax(np.cumsum(SigS) / SigS_sum >= np.random.rand())

                else:  # absorption

                    absorbed = True

                    # Neutron is converted to the new fission neutron with
                    # the weight increased by eta
                    weight[iNeutron] *= SigP / SigA

                    # Sample the energy group for the new-born neutron
                    iGroup[iNeutron] = np.argmax(np.cumsum(fuel['chi']) >= np.random.rand())

        # End of neutron random walk cycle: from emission to absorption
    # End of loop over neutrons
    #-------------------------------------------------------------------------------------------
    # Russian roulette
    weight = russian_roulette(weight, weight0)

    #-------------------------------------------------------------------------------------------
    # Clean up absorbed or killed neutrons
    indices = np.nonzero(weight)
    x = x[indices]
    y = y[indices]
    iGroup = iGroup[indices]
    weight = weight[indices]
    numNeutrons = weight.shape[0]

    #-------------------------------------------------------------------------------------------
    # Split too "heavy" neutrons
    weight, numNeutrons, x, y, iGroup = split_neutrons(weight, numNeutrons, x, y, iGroup)

    #-------------------------------------------------------------------------------------------
    # k-eff in a cycle equals the total weight of the new generation over
    # the total weight of the old generation (the old generation weight =
    # numNeutronsBorn)
    keff_cycle = np.sum(weight) / np.sum(weight0)

    iActive = iCycle - numCycles_inactive
    if iActive <= 0:
        print('Inactive cycle = {:3d}/{:3d}; k-eff cycle = {:.5f}; numNeutrons = {:3d}'.format(
            iCycle, numCycles_inactive, keff_cycle, numNeutrons))
    else:
        print("iActive =", iActive)
        # k-effective of the cycle
        keff_active_cycle[iActive-1] = keff_cycle

        # k-effective of the problem
        keff_expected[iActive-1] = np.mean(keff_active_cycle[:iActive])

        # Standard deviation of k-effective
        sigma_keff[iActive-1] = np.sqrt(
            np.sum((keff_active_cycle[:iActive] - keff_expected[iActive-1]) ** 2) / max(iActive - 1, 1) / iActive)

        print('Active cycle = {:3d}/{:3d}; k-eff cycle = {:.5f}; numNeutrons = {:3d}; k-eff expected = {:.5f}; sigma = {:.5f}'.format(
            iCycle - numCycles_inactive, numCycles_active, keff_cycle, numNeutrons, keff_expected[iActive-1], sigma_keff[iActive-1]))

    # End of main (power) iteration
"""

'# Main (power) iteration loop\nfor iCycle in range(1, numCycles_inactive + numCycles_active + 1):\n\n    # Normalize the weights of the neutrons to make the total weight equal to\n    # numNeutrons_born (equivalent to division by keff_cycle)\n    weight = (weight / np.sum(weight, axis=0, keepdims=True)) * numNeutrons_born\n    weight0 = weight.copy()\n    #----------------------------------------------------------------------\n    # Loop over neutrons\n    for iNeutron in range(numNeutrons):\n\n        absorbed = False\n\n        #------------------------------------------------------------------\n        # Neutron random walk cycle: from emission to absorption\n\n        while not absorbed:\n\n            # Sample free path length according to the Woodcock method\n            freePath = -np.log(np.random.rand()) / SigTmax[iGroup[iNeutron]]\n            #print(freePath)\n\n            if not virtualCollision:\n                # Sample the direction of neutron flight assuming both\n   

In [21]:
"""#@njit
def perform_random_walk(x, y, iGroup, weight, numNeutrons, numNeutrons_born, numCycles_inactive, numCycles_active, virtualCollision):
    # Define other variables and arrays used in the code

    # Main (power) iteration loop
    for iCycle in range(1, numCycles_inactive + numCycles_active + 1):

        # Normalize the weights of the neutrons to make the total weight equal to
        # numNeutrons_born (equivalent to division by keff_cycle)
        weight = (weight / np.sum(weight, axis=0, keepdims=True)) * numNeutrons_born
        weight0 = weight.copy()

        # Loop over neutrons
        for iNeutron in range(numNeutrons):
            absorbed = False

            # Neutron random walk cycle: from emission to absorption
            while not absorbed:
                # Sample free path length according to the Woodcock method
                freePath = -np.log(np.random.rand()) / SigTmax[iGroup[iNeutron]]
            #print(freePath)

            if not virtualCollision:
                # Sample the direction of neutron flight assuming both
                # fission and scattering are isotropic in the lab (a strong
                # assumption!)
                teta = np.pi * np.random.rand()
                phi = 2.0 * np.pi * np.random.rand()
                dirX = np.sin(teta) * np.cos(phi)
                dirY = np.sin(teta) * np.sin(phi)

            # Fly
            x[iNeutron] += freePath * dirX
            y[iNeutron] += freePath * dirY

            # If outside the cell, find the corresponding point inside the
            # cell
            while x[iNeutron] < 0:
                x[iNeutron] += pitch
            while y[iNeutron] < 0:
                y[iNeutron] += pitch
            while x[iNeutron] > pitch:
                x[iNeutron] -= pitch
            while y[iNeutron] > pitch:
                y[iNeutron] -= pitch

            # Find the total and scattering cross sections
            if 0.9 < x[iNeutron] < 2.7:  # INPUT
                SigA = fuelTuple[1][iGroup[iNeutron]] + fuelTuple[0][iGroup[iNeutron]] + fuelTuple[2][iGroup[iNeutron]]
                SigS = fuelTuple[-2][-2][iGroup[iNeutron], :].reshape(-1, 1)
                SigP = fuelTuple[3][0, iGroup[iNeutron]]
            elif x[iNeutron] < 0.7 or x[iNeutron] > 2.9:  # INPUT
                SigA = coolTuple[0][iGroup[iNeutron]] + coolTuple[2][iGroup[iNeutron]]
                SigS = coolTuple[-2][-3][iGroup[iNeutron], :].reshape(-1, 1)
                SigP = 0
            else:
                SigA = cladTuple[0][iGroup[iNeutron]] + cladTuple[2][iGroup[iNeutron]]
                SigS = cladTuple[-2][-3][iGroup[iNeutron], :].reshape(-1, 1)
                SigP = 0

            # Find the other cross sections ...
            # ... scattering
            SigS_sum = np.sum(SigS)
            # ... total
            SigT = SigA + SigS_sum
            # ... virtual
            SigV = SigTmax[iGroup[iNeutron]] - SigT

            # Sample the type of the collision: virtual (do nothing) or real
            if SigV / SigTmax[iGroup[iNeutron]] >= np.random.rand():  # virtual collision

                virtualCollision = True

            else:  # real collision

                virtualCollision = False

                # Sample type of the collision: scattering or absorption
                if SigS_sum / SigT >= np.random.rand():  # isotropic scattering

                    # Score scatterings with account for weight divided by the
                    # total scattering cross section
                    detectS[iGroup[iNeutron]] += weight[iNeutron] / SigS_sum

                    # Sample the energy group of the secondary neutron
                    iGroup[iNeutron] = np.argmax(np.cumsum(SigS) / SigS_sum >= np.random.rand())

                else:  # absorption

                    absorbed = True

                    # Neutron is converted to the new fission neutron with
                    # the weight increased by eta
                    weight[iNeutron] *= SigP / SigA

                    # Sample the energy group for the new-born neutron
                    iGroup[iNeutron] = np.argmax(np.cumsum(fuelTuple[6]) >= np.random.rand())

        # End of neutron random walk cycle: from emission to absorption
    # End of loop over neutrons
    #-------------------------------------------------------------------------------------------
    # Russian roulette
    for iNeutron in range(numNeutrons):
        terminateP = 1 - weight[iNeutron] / weight0[iNeutron]
        if terminateP >= np.random.rand():
            weight[iNeutron] = 0  # killed
        elif terminateP > 0:
            weight[iNeutron] = weight0[iNeutron]  # restore the weight

    #-------------------------------------------------------------------------------------------
    # Clean up absorbed or killed neutrons
    indices = np.nonzero(weight)
    x = x[indices]
    y = y[indices]
    iGroup = iGroup[indices]
    weight = weight[indices]
    numNeutrons = weight.shape[0]

    #-------------------------------------------------------------------------------------------
    # Split too "heavy" neutrons
    numNew = 0
    for iNeutron in range(numNeutrons):
        if weight[iNeutron] > 1:
            # Truncated integer value of the neutron weight
            N = int(np.floor(weight[iNeutron]))
            # Sample the number of split neutrons
            if weight[iNeutron] - N > np.random.rand():
                N += 1
            # Change the weight of the split neutron
            weight[iNeutron] = weight[iNeutron] / N
            # Introduce new neutrons
            for iNew in range(N - 1):
                numNew += 1
                x = np.append(x, x[iNeutron])
                y = np.append(y, y[iNeutron])
                weight = np.append(weight, weight[iNeutron])
                iGroup = np.append(iGroup, iGroup[iNeutron])

    # Increase the number of neutrons
    numNeutrons += numNew

    #-------------------------------------------------------------------------------------------
    # k-eff in a cycle equals the total weight of the new generation over
    # the total weight of the old generation (the old generation weight =
    # numNeutronsBorn)
    keff_cycle = np.sum(weight) / np.sum(weight0)

    iActive = iCycle - numCycles_inactive
    if iActive <= 0:
        print('Inactive cycle = {:3d}/{:3d}; k-eff cycle = {:.5f}; numNeutrons = {:3d}'.format(
            iCycle, numCycles_inactive, keff_cycle, numNeutrons))
    else:
        print("iActive =", iActive)
        # k-effective of the cycle
        keff_active_cycle[iActive-1] = keff_cycle

        # k-effective of the problem
        keff_expected[iActive-1] = np.mean(keff_active_cycle[:iActive])

        # Standard deviation of k-effective
        sigma_keff[iActive-1] = np.sqrt(
            np.sum((keff_active_cycle[:iActive] - keff_expected[iActive-1]) ** 2) / max(iActive - 1, 1) / iActive)

        print('Active cycle = {:3d}/{:3d}; k-eff cycle = {:.5f}; numNeutrons = {:3d}; k-eff expected = {:.5f}; sigma = {:.5f}'.format(
            iCycle - numCycles_inactive, numCycles_active, keff_cycle, numNeutrons, keff_expected[iActive-1], sigma_keff[iActive-1]))

    # End of main (power) iteration

perform_random_walk(x, y, iGroup, weight, numNeutrons, numNeutrons_born, numCycles_inactive, numCycles_active, virtualCollision)"""

'#@njit\ndef perform_random_walk(x, y, iGroup, weight, numNeutrons, numNeutrons_born, numCycles_inactive, numCycles_active, virtualCollision):\n    # Define other variables and arrays used in the code\n\n    # Main (power) iteration loop\n    for iCycle in range(1, numCycles_inactive + numCycles_active + 1):\n\n        # Normalize the weights of the neutrons to make the total weight equal to\n        # numNeutrons_born (equivalent to division by keff_cycle)\n        weight = (weight / np.sum(weight, axis=0, keepdims=True)) * numNeutrons_born\n        weight0 = weight.copy()\n\n        # Loop over neutrons\n        for iNeutron in range(numNeutrons):\n            absorbed = False\n\n            # Neutron random walk cycle: from emission to absorption\n            while not absorbed:\n                # Sample free path length according to the Woodcock method\n                freePath = -np.log(np.random.rand()) / SigTmax[iGroup[iNeutron]]\n            #print(freePath)\n\n            if n

In [22]:
"""np.random.seed(42)  # Set the seed to 42
freePath = -np.log(np.random.rand()) / SigTmax[iGroup[iNeutron]]
np.random.seed(42)  # Set the seed to 42
print(np.random.rand())
print(iGroup[iNeutron])
print(SigTmax[iGroup[iNeutron]])
print(freePath)"""

'np.random.seed(42)  # Set the seed to 42\nfreePath = -np.log(np.random.rand()) / SigTmax[iGroup[iNeutron]]\nnp.random.seed(42)  # Set the seed to 42\nprint(np.random.rand())\nprint(iGroup[iNeutron])\nprint(SigTmax[iGroup[iNeutron]])\nprint(freePath)'

### The Default Main iteration loop (no functions)

In [23]:
"""# Main (power) iteration loop
for iCycle in range(1, numCycles_inactive + numCycles_active + 1):

    # Normalize the weights of the neutrons to make the total weight equal to
    # numNeutrons_born (equivalent to division by keff_cycle)
    weight = (weight / np.sum(weight, axis=0, keepdims=True)) * numNeutrons_born
    weight0 = weight.copy()
    #----------------------------------------------------------------------
    # Loop over neutrons
    for iNeutron in range(numNeutrons):

        absorbed = False

        #------------------------------------------------------------------
        # Neutron random walk cycle: from emission to absorption

        while not absorbed:

            # Sample free path length according to the Woodcock method
            freePath = -np.log(np.random.rand()) / SigTmax[iGroup[iNeutron]]
            #print(freePath)

            if not virtualCollision:
                # Sample the direction of neutron flight assuming both
                # fission and scattering are isotropic in the lab (a strong
                # assumption!)
                teta = np.pi * np.random.rand()
                phi = 2.0 * np.pi * np.random.rand()
                dirX = np.sin(teta) * np.cos(phi)
                dirY = np.sin(teta) * np.sin(phi)

            # Fly
            x[iNeutron] += freePath * dirX
            y[iNeutron] += freePath * dirY

            # If outside the cell, find the corresponding point inside the
            # cell
            while x[iNeutron] < 0:
                x[iNeutron] += pitch
            while y[iNeutron] < 0:
                y[iNeutron] += pitch
            while x[iNeutron] > pitch:
                x[iNeutron] -= pitch
            while y[iNeutron] > pitch:
                y[iNeutron] -= pitch

            # Find the total and scattering cross sections
            if 0.9 < x[iNeutron] < 2.7:  # INPUT
                SigA = fuel['SigF'][iGroup[iNeutron]] + fuel['SigC'][iGroup[iNeutron]] + fuel['SigL'][iGroup[iNeutron]]
                SigS = fuel["sigS_G"]["sparse_SigS[0]"][iGroup[iNeutron], :].reshape(-1, 1)
                SigP = fuel['SigP'][0, iGroup[iNeutron]]
            elif x[iNeutron] < 0.9 or x[iNeutron] > 2.7:  # INPUT
                SigA = cool['SigC'][iGroup[iNeutron]] + cool['SigL'][iGroup[iNeutron]]
                SigS = cool["sigS_G"]["sparse_SigS[0]"][iGroup[iNeutron], :].reshape(-1, 1)
                SigP = 0
            else:
                SigA = clad['SigC'][iGroup[iNeutron]] + clad['SigL'][iGroup[iNeutron]]
                SigS = clad["sigS_G"]["sparse_SigS[0]"][iGroup[iNeutron], :].reshape(-1, 1)
                SigP = 0

            # Find the other cross sections ...
            # ... scattering
            SigS_sum = np.sum(SigS)
            # ... total
            SigT = SigA + SigS_sum
            # ... virtual
            SigV = SigTmax[iGroup[iNeutron]] - SigT

            # Sample the type of the collision: virtual (do nothing) or real
            if SigV / SigTmax[iGroup[iNeutron]] >= np.random.rand():  # virtual collision

                virtualCollision = True

            else:  # real collision

                virtualCollision = False

                # Sample type of the collision: scattering or absorption
                if SigS_sum / SigT >= np.random.rand():  # isotropic scattering

                    # Score scatterings with account for weight divided by the
                    # total scattering cross section
                    detectS[iGroup[iNeutron]] += weight[iNeutron] / SigS_sum

                    # Sample the energy group of the secondary neutron
                    iGroup[iNeutron] = np.argmax(np.cumsum(SigS) / SigS_sum >= np.random.rand())

                else:  # absorption

                    absorbed = True

                    # Neutron is converted to the new fission neutron with
                    # the weight increased by eta
                    weight[iNeutron] *= SigP / SigA

                    # Sample the energy group for the new-born neutron
                    iGroup[iNeutron] = np.argmax(np.cumsum(fuel['chi']) >= np.random.rand())

        # End of neutron random walk cycle: from emission to absorption
    # End of loop over neutrons
    #-------------------------------------------------------------------------------------------
    # Russian roulette
    for iNeutron in range(numNeutrons):
        terminateP = 1 - weight[iNeutron] / weight0[iNeutron]
        if terminateP >= np.random.rand():
            weight[iNeutron] = 0  # killed
        elif terminateP > 0:
            weight[iNeutron] = weight0[iNeutron]  # restore the weight

    #-------------------------------------------------------------------------------------------
    # Clean up absorbed or killed neutrons
    indices = np.nonzero(weight)
    x = x[indices]
    y = y[indices]
    iGroup = iGroup[indices]
    weight = weight[indices]
    numNeutrons = weight.shape[0]

    #-------------------------------------------------------------------------------------------
    # Split too "heavy" neutrons
    numNew = 0
    for iNeutron in range(numNeutrons):
        if weight[iNeutron] > 1:
            # Truncated integer value of the neutron weight
            N = int(np.floor(weight[iNeutron]))
            # Sample the number of split neutrons
            if weight[iNeutron] - N > np.random.rand():
                N += 1
            # Change the weight of the split neutron
            weight[iNeutron] = weight[iNeutron] / N
            # Introduce new neutrons
            for iNew in range(N - 1):
                numNew += 1
                x = np.append(x, x[iNeutron])
                y = np.append(y, y[iNeutron])
                weight = np.append(weight, weight[iNeutron])
                iGroup = np.append(iGroup, iGroup[iNeutron])

    # Increase the number of neutrons
    numNeutrons += numNew

    #-------------------------------------------------------------------------------------------
    # k-eff in a cycle equals the total weight of the new generation over
    # the total weight of the old generation (the old generation weight =
    # numNeutronsBorn)
    keff_cycle = np.sum(weight) / np.sum(weight0)

    iActive = iCycle - numCycles_inactive
    if iActive <= 0:
        print('Inactive cycle = {:3d}/{:3d}; k-eff cycle = {:.5f}; numNeutrons = {:3d}'.format(
            iCycle, numCycles_inactive, keff_cycle, numNeutrons))
    else:
        print("iActive =", iActive)
        # k-effective of the cycle
        keff_active_cycle[iActive-1] = keff_cycle

        # k-effective of the problem
        keff_expected[iActive-1] = np.mean(keff_active_cycle[:iActive])

        # Standard deviation of k-effective
        sigma_keff[iActive-1] = np.sqrt(
            np.sum((keff_active_cycle[:iActive] - keff_expected[iActive-1]) ** 2) / max(iActive - 1, 1) / iActive)

        print('Active cycle = {:3d}/{:3d}; k-eff cycle = {:.5f}; numNeutrons = {:3d}; k-eff expected = {:.5f}; sigma = {:.5f}'.format(
            iCycle - numCycles_inactive, numCycles_active, keff_cycle, numNeutrons, keff_expected[iActive-1], sigma_keff[iActive-1]))

    # End of main (power) iteration
"""

'# Main (power) iteration loop\nfor iCycle in range(1, numCycles_inactive + numCycles_active + 1):\n\n    # Normalize the weights of the neutrons to make the total weight equal to\n    # numNeutrons_born (equivalent to division by keff_cycle)\n    weight = (weight / np.sum(weight, axis=0, keepdims=True)) * numNeutrons_born\n    weight0 = weight.copy()\n    #----------------------------------------------------------------------\n    # Loop over neutrons\n    for iNeutron in range(numNeutrons):\n\n        absorbed = False\n\n        #------------------------------------------------------------------\n        # Neutron random walk cycle: from emission to absorption\n\n        while not absorbed:\n\n            # Sample free path length according to the Woodcock method\n            freePath = -np.log(np.random.rand()) / SigTmax[iGroup[iNeutron]]\n            #print(freePath)\n\n            if not virtualCollision:\n                # Sample the direction of neutron flight assuming both\n   

### The main iteration loop (with functions)

In [24]:
# Start stopwatch
start_time = t.time()  # Placeholder for stopwatch functionality in Python

#--------------------------------------------------------------------------
# Number of source neutrons
numNeutrons_born = 100      # INPUT

# Number of inactive source cycles to skip before starting k-eff accumulation
numCycles_inactive = 100    # INPUT

# Number of active source cycles for k-eff accumulation
numCycles_active = 100     # INPUT

# Size of the square unit cell
pitch = 3.6  # cm           # INPUT

# Define fuel and coolant regions
fuelLeft, fuelRight = 0.9, 2.7  # INPUT
coolLeft, coolRight = 0.7, 2.9  # INPUT
#--------------------------------------------------------------------------
# Path to macroscopic cross section data:
# (Assuming the corresponding data files are available and accessible)
#macro_xs_path = '..//02.Macro.XS.421g'

# Fill the structures fuel, clad, and cool with the cross-section data
fuel = hdf2dict('macro421_UO2_03__900K.h5')  # INPUT
print(f"File 'macro421_UO2_03__900K.h5' has been read in.")
clad = hdf2dict('macro421_Zry__600K.h5')     # INPUT
print(f"File 'macro421_Zry__600K.h5' has been read in.")
cool  = hdf2dict('macro421_H2OB__600K.h5')   # INPUT
print(f"File 'macro421_H2OB__600K.h5' has been read in.")

# Define the majorant: the maximum total cross-section vector
# This part just looks at all the three different vectors and creates a new
# vector that picks out the largest value for every index from each of the 
# three vectros. Althought there are some rounding differences.
SigTmax = np.max(np.vstack((fuel["SigT"], clad["SigT"], cool["SigT"])), axis=0)

# Number of energy groups
ng = fuel['ng']

#--------------------------------------------------------------------------
# Detectors
detectS = np.zeros(ng)

#--------------------------------------------------------------------------
# Four main vectors describing the neutrons in a batch
x = np.zeros(numNeutrons_born * 2)
y = np.zeros(numNeutrons_born * 2)
weight = np.ones(numNeutrons_born * 2)
iGroup = np.ones(numNeutrons_born * 2, dtype=int)

#--------------------------------------------------------------------------
# Neutrons are assumed born randomly distributed in the cell with weight 1
# with sampled fission energy spectrum
numNeutrons = numNeutrons_born
for iNeutron in range(numNeutrons):
    x[iNeutron] = np.random.rand() * pitch
    y[iNeutron] = np.random.rand() * pitch
    weight[iNeutron] = 1
    # Sample the neutron energy group
    iGroup[iNeutron] = np.argmax(np.cumsum(fuel['chi']) >= np.random.rand())
#print(iGroup)

#--------------------------------------------------------------------------
# Prepare vectors for keff and standard deviation of keff
keff_expected = np.ones(numCycles_active)
sigma_keff = np.zeros(numCycles_active)
keff_active_cycle = np.ones(numCycles_active)
virtualCollision = False

# Main (power) iteration loop
for iCycle in range(1, numCycles_inactive + numCycles_active + 1):

    # Normalize the weights of the neutrons to make the total weight equal to
    # numNeutrons_born (equivalent to division by keff_cycle)
    #weight = (weight / np.sum(weight, axis=0, keepdims=True)) * numNeutrons_born
    weight = (weight / np.sum(weight)) * numNeutrons_born
    weight0 = weight.copy()
    #print("weight0 = ", weight0)
    #----------------------------------------------------------------------
    # Loop over neutrons
    for iNeutron in range(numNeutrons):

        absorbed = False

        #------------------------------------------------------------------
        # Neutron random walk cycle: from emission to absorption

        while not absorbed:
            #print("Neutron was absorbed!")
            # Sample free path length according to the Woodcock method
            freePath = -np.log(np.random.rand()) / SigTmax[iGroup[iNeutron]]
            #print("freePath = ", freePath)

            if not virtualCollision:
                #print("Collision was virtual!")
                # Sample the direction of neutron flight assuming both
                # fission and scattering are isotropic in the lab (a strong
                # assumption!)
                dirX, dirY = sample_direction()
            # Fly
            x, y = move_neutron(x, y, iNeutron, pitch, freePath, dirX, dirY)

            # Find the total and scattering cross sections                    
            SigA, SigS, SigP = calculate_cross_sections(fuelLeft, fuelRight, coolLeft, coolRight, x, iNeutron, iGroup, fuel, cool, clad)
            
            # Sample the type of the collision: virtual (do nothing) or real
            absorbed, virtualCollision, iGroup, weight, detectS = perform_collision(virtualCollision, absorbed, SigS, SigA, SigP, 
                                                                                SigTmax, iGroup, iNeutron, detectS, weight, fuel["chi"])
        # End of neutron random walk cycle: from emission to absorption
    # End of loop over neutrons
    #-------------------------------------------------------------------------------------------
    # Russian roulette
    weight = russian_roulette(weight, weight0)

    #-------------------------------------------------------------------------------------------
    # Clean up absorbed or killed neutrons
    x, y, iGroup, weight, numNeutrons = update_indices(x, y, iGroup, weight)

    #-------------------------------------------------------------------------------------------
    # Split too "heavy" neutrons
    weight, numNeutrons, x, y, iGroup = split_neutrons(weight, numNeutrons, x, y, iGroup)

    #-------------------------------------------------------------------------------------------
    # k-eff in a cycle equals the total weight of the new generation over
    # the total weight of the old generation (the old generation weight =
    # numNeutronsBorn)
    calculate_keff_cycle(iCycle, numCycles_inactive, numCycles_active, weight, weight0, numNeutrons, keff_active_cycle, keff_expected, sigma_keff)

    # End of main (power) iteration

File 'macro421_UO2_03__900K.h5' has been read in.
File 'macro421_Zry__600K.h5' has been read in.
File 'macro421_H2OB__600K.h5' has been read in.
Inactive cycle =   1/100; k-eff cycle = 0.94566; numNeutrons = 159
Inactive cycle =   2/100; k-eff cycle = 1.21925; numNeutrons = 130
Inactive cycle =   3/100; k-eff cycle = 0.98118; numNeutrons = 108
Inactive cycle =   4/100; k-eff cycle = 0.96380; numNeutrons = 101
Inactive cycle =   5/100; k-eff cycle = 1.03612; numNeutrons = 105
Inactive cycle =   6/100; k-eff cycle = 1.21315; numNeutrons = 118
Inactive cycle =   7/100; k-eff cycle = 1.21048; numNeutrons = 125
Inactive cycle =   8/100; k-eff cycle = 1.12001; numNeutrons = 118
Inactive cycle =   9/100; k-eff cycle = 1.14792; numNeutrons = 117
Inactive cycle =  10/100; k-eff cycle = 0.97439; numNeutrons =  99
Inactive cycle =  11/100; k-eff cycle = 1.03447; numNeutrons = 107
Inactive cycle =  12/100; k-eff cycle = 1.03386; numNeutrons = 109
Inactive cycle =  13/100; k-eff cycle = 1.13478; nu

In [28]:
len(keff_expected)

100

In [ ]:
fuel.keys()

In [ ]:
from numba import njit
import numpy as np

@njit
def process_fuel_data(fuel_data: tuple) -> np.ndarray:
    sigC, sigF, sigL, chi = fuel_data

    # Perform computations using the fuel data
    # Example: Compute the total cross section
    sigT = sigC + sigF + sigL

    # Return the computed value
    return sigT

# Test data
testdata = (
    np.array([0.5, 0.3, 0.2]),
    np.array([0.1, 0.2, 0.3]),
    np.array([0.4, 0.2, 0.1]),
    np.array([1.0, 0.8, 0.5])
)

# Call the function
result = process_fuel_data(testdata)
print(type(result))



In [ ]:
type(numNeutrons)